# Sports Sentiment Analysis Final Project

by Mikey McGrath

## 1. Library Imports

In [33]:
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [34]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mikeymcgrath/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mikeymcgrath/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mikeymcgrath/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## 2. Data Processing

This data is only from September of the previous season. I can access the other months of other seasons as well. 

In [ ]:
# Importing the html file and seeing what it looks like
f = open("data.html", "rb")
doc = BeautifulSoup(f, "html.parser")
doc.prettify()
doc

In [121]:
# List of NFL teams
teams = {"Bills" : [],"Dolphins": [],"Patriots": [],"Jets": [],
			"Ravens": [],"Bengals": [],"Browns": [],"Steelers": [],
			"Texans": [],"Colts": [],"Jaguars": [],"Titans": [],
			"Broncos": [],"Chiefs": [],"Raiders": [],"Chargers": [],
			"Cowboys": [],"Giants": [],"Eagles": [],"Commanders": [],
			"Bears": [],"Lions": [],"Packers": [],"Vikings": [],
			"Falcons": [],"Panthers": [],"Saints": [],"Buccaneers": [],
			"Cardinals": [],"Rams": [],"49ers": [],"Seahawks": []}
team_scores = {"Bills" : [],"Dolphins": [],"Patriots": [],"Jets": [],
			"Ravens": [],"Bengals": [],"Browns": [],"Steelers": [],
			"Texans": [],"Colts": [],"Jaguars": [],"Titans": [],
			"Broncos": [],"Chiefs": [],"Raiders": [],"Chargers": [],
			"Cowboys": [],"Giants": [],"Eagles": [],"Commanders": [],
			"Bears": [],"Lions": [],"Packers": [],"Vikings": [],
			"Falcons": [],"Panthers": [],"Saints": [],"Buccaneers": [],
			"Cardinals": [],"Rams": [],"49ers": [],"Seahawks": []}

In [122]:
headlines = []
for link in doc.find_all("a"):
    text = link.get("title")
    if text is not None:
        # Getting rid of headlines that don't mention a team
        for team in teams:
            if team in text and text not in headlines:
                headlines.append(text)


maybe find a way to include the cities in case the mascot is not a part of the headline

In [123]:
# Assign each team their headlines
for team in teams:
    for headline in headlines:
        if team in headline:
            teams[team].append(headline)


## 3. Sentiment Analysis

In [124]:
sia = SentimentIntensityAnalyzer()

In [125]:
# Return the sentiment score of each headline
def get_polarity_scores(headline):
    scores = sia.polarity_scores(headline)
    return scores['neg'], scores['pos'], scores['neu'], scores['compound']

In [126]:
# WORK ON FLUENCY OF THIS CHUNK
def get_avg_polarity_scores(team):
    neg_sum = 0
    pos_sum = 0
    neu_sum = 0
    compound_sum = 0
    for headline in teams[team]:
        scores = get_polarity_scores(headline)
        neg_sum += scores[0]
        pos_sum += scores[1]
        neu_sum += scores[2]
        compound_sum += scores[3]
    num_headlines = len(teams[team])
    neg = neg_sum / num_headlines
    pos = pos_sum / num_headlines
    neu = neu_sum / num_headlines
    compound = compound_sum / num_headlines
    return (neg, pos, neu, compound)

In [ ]:
for team in teams:
    team_scores[team].append(get_avg_polarity_scores(team))
    # FIGURE OUT WHY. I NEED TO DO THIS (SHOULDN"T HAVE TO AND IT messes THiNGS UP)
    team_scores[team] = team_scores[team][0]
team_scores

## 4. Ranking Teams